In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

2024-11-15 20:44:30.513360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 20:44:30.919800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 20:44:32.798103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/maverick/Downloads/courses/cs685/ProjectWorkspace/models/modelenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:



testedModels = [
    # "bhadresh-savani/bert-base-go-emotion",
    "SchuylerH/bert-multilingual-go-emtions",
    # "joeddav/distilbert-base-uncased-go-emotions-student",
    # "SamLowe/roberta-base-go_emotions",
    # # below two are good models, can refer documentation for more details
    # "michellejieli/emotion_text_classifier",
    # "j-hartmann/emotion-english-distilroberta-base",
    # "gokuls/BERT-tiny-emotion-intent"
]

models = {}
def LoadModels():
    # 1. Load Pre-trained Model for Emotion Classification
    # loading all models in for loop
    for model_name in testedModels:
        print(f"Loading model: {model_name}...")
        
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        
        # Move model to GPU if available
        if torch.cuda.is_available():
            print(f"Model {model_name} loaded on GPU.")
            model.to("cuda")
        else:
            print(f"GPU not available. Model {model_name} will run on CPU.")
        

        
        # High-level pipeline with GPU enabled
        classifier = pipeline(
            "text-classification",
            model=model,
            tokenizer=tokenizer,
            device=0 if torch.cuda.is_available() else -1,  # Device 0 for GPU, -1 for CPU
            return_all_scores=True,
        )
        
        models[model_name] = {"classifier": classifier}
LoadModels()    


Loading model: SchuylerH/bert-multilingual-go-emtions...
Model SchuylerH/bert-multilingual-go-emtions loaded on GPU.


/home/maverick/Downloads/courses/cs685/ProjectWorkspace/models/modelenv/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
import clean
%load_ext autoreload
%autoreload 2
# file = 'data/twitter-stream-2021-06-23_filtered.json'

# data1 = []
# data2 = []

# with open(file) as f:
#     for line in f:
#         try:
#             texts = json.loads(line)['output']['Objects']
#             for i in range(0, len(texts)):
#                 data1.append(texts[i]['text'])
#                 data2.append(clean.preprocess_tweet(texts[i]['text']))
#         except json.JSONDecodeError as e:
#             print(f"Error decoding JSON in {file}: {e}")

# # print(data)

# # redirect data to json format
# with open('data/dump_withStopWords.json', 'w') as outfile:
#     for entry in data2:
#         json.dump(entry, outfile)
#         outfile.write('\n')

# with open('data/dump.json', 'w') as outfile:
#     for entry in data1:
#         json.dump(entry, outfile)
#         outfile.write('\n')
# print('File written')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maverick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maverick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# take two lines at a time from both files

# create folder data_cleaned if it doesn't exist
import os
# if not os.path.exists('data_cleaned'):
#     os.makedirs('data_cleaned')

# input_dir = 'New'
# output_dir = 'data_cleaned'


# for file in os.listdir(input_dir):
#     with open(os.path.join(input_dir, file)) as f1:
#         with open(os.path.join(output_dir, file), 'w') as f2:
#             for line1 in f1:
#                 # get the text from the line
#                 text1 = json.loads(line1)
#                 new_twts = []

#                 for twt in text1['output']['Objects']:
#                     new_twt = twt.copy()
#                     new_twt['text'] = clean.preprocess_tweet(twt['text'])
#                     new_twts.append(new_twt)
                
#                 text1['output']['Objects'] = new_twts

#                 # dump this new text to a new file
#                 json.dump(text1, f2)
#                 f2.write('\n')


In [26]:
# all the categories of emotions
# Emotion to class name mapping
# Emotion to category mapping
emotion_to_category = {
    "admiration": "Positive or Joyful Emotions",
    "amusement": "Positive or Joyful Emotions",
    "joy": "Positive or Joyful Emotions",
    "gratitude": "Positive or Joyful Emotions",
    "excitement": "Positive or Joyful Emotions",
    "optimism": "Positive or Joyful Emotions",
    "pride": "Positive or Joyful Emotions",
    "relief": "Positive or Joyful Emotions",
    "anger": "Negative or Fearful Emotions",
    "annoyance": "Negative or Fearful Emotions",
    "fear": "Negative or Fearful Emotions",
    "nervousness": "Negative or Fearful Emotions",
    "sadness": "Negative or Fearful Emotions",
    "grief": "Negative or Fearful Emotions",
    "remorse": "Negative or Fearful Emotions",
    "neutral": "Neutral or Objective Emotions",
    "approval": "Neutral or Objective Emotions",
    "realization": "Neutral or Objective Emotions",
    "love": "Love and Caring Emotions",
    "caring": "Love and Caring Emotions",
    "desire": "Love and Caring Emotions",
    "surprise": "Surprise or Unexpected Responses",
    "curiosity": "Surprise or Unexpected Responses",
    "confusion": "Surprise or Unexpected Responses",
    "disappointment": "Displeasure or Disapproval Emotions",
    "disapproval": "Displeasure or Disapproval Emotions",
    "disgust": "Displeasure or Disapproval Emotions",
    "embarrassment": "Displeasure or Disapproval Emotions"
}

# Category to emotions mapping
category_to_emotions = {
    "Positive or Joyful Emotions": [
        "admiration", "amusement", "joy", "gratitude", "excitement", 
        "optimism", "pride", "relief"
    ],
    "Negative or Fearful Emotions": [
        "anger", "annoyance", "fear", "nervousness", "sadness", 
        "grief", "remorse"
    ],
    "Neutral or Objective Emotions": [
        "neutral", "approval", "realization"
    ],  
    "Love and Caring Emotions": [
        "love", "caring", "desire"
    ],
    "Surprise or Unexpected Responses": [
        "surprise", "curiosity", "confusion"
    ],
    "Displeasure or Disapproval Emotions": [
        "disappointment", "disapproval", "disgust","embarrassment"
    ],
}


# change to parent emotion function from single result obj
def change_to_parent_emotion(res):
    res['label'] = emotion_to_category[res['label']]
    return res


In [19]:
# # two files, dump_withStopWords.json and dump_noStopWords.json
# # both contain the real tweet, and the tweet after preprocessing, make table of results for each models


# # 3. Classify Emotions
# results = {}
inputdir = 'data_cleaned'
# file = 'twitter-stream-2021-06-23_filtered.json'
outputdir = 'data_with_emotions'

# # for model_name in models:
# model_name = testedModels[0]

# # create folder data_with_emotions if it doesn't exist
# if not os.path.exists(outputdir):
#     os.makedirs(outputdir)

# # take two lines at a time from both files
classifier = models[model_name]["classifier"]
# # Process files
# batch_results = []
# for file in os.listdir(inputdir):
#     if file in os.listdir(outputdir):
#         print(f"File {file} already processed")
#         continue
#     with open(os.path.join(inputdir, file)) as f1:
#         # Collect all texts from the input file for batch processing
#         texts = []
#         text_count = []
#         tcount = 0
#         for line in f1:
#             text_entry = json.loads(line)
#             count = 0
#             for obj in text_entry['output']['Objects']:
#                 texts.append(obj["text"])
#                 count += 1
#             text_count.append(count)
#             tcount += 1

#         # Batch process the texts to classify emotions
#         batch_results = classifier(texts, batch_size=16)  # Adjust batch size based on GPU memory

#         # print(len(text_count))
#         # print(tcount)
#     with open(os.path.join(inputdir, file)) as f1, open(os.path.join(outputdir, file), 'w') as f2:
        
#         # Update the original data with emotion results
#         text_index = 0
#         # lineCount = 0
#         for line, count in zip(f1, text_count):
#             text_entry = json.loads(line)
#             for obj in text_entry['output']['Objects']:
#                 obj["emotion"] = batch_results[text_index]
#                 # print(batch_results[text_index])
#                 text_index += 1
#             # Write the updated data back to the output file
#             f2.write(json.dumps(text_entry) + '\n')
#         # Write the updated data back to the output file
#     print(f"File {file} written")
#     # break

File twitter_stream_2020_01_07_filtered.json already processed
File twitter-stream-2021-05-24_filtered.json already processed
File twitter-stream-20211004_filtered.json already processed
File twitter-stream-2020-09-22_filtered.json already processed
File twitter-stream-2020-11-21_filtered.json already processed
File twitter-stream-2020-07-12_filtered.json already processed
File twitter-stream-2020-12-21_filtered.json already processed
File twitter-stream-20211028_filtered.json already processed
File twitter-stream-2021-02-25_filtered.json already processed
File twitter-stream-2020-10-16_filtered.json already processed
File twitter-stream-2021-03-03_filtered.json already processed
File twitter-stream-2021-07-05_filtered.json already processed
File twitter-stream-2020-12-03_filtered.json already processed
File twitter-stream-20211022_filtered.json already processed
File twitter-stream-20211127_filtered.json already processed
File twitter-stream-2021-01-26_filtered.json already processed


KeyboardInterrupt: 

In [23]:
from concurrent.futures import ThreadPoolExecutor



def process_file(file):
    """Process a single file to classify emotions."""
    input_path = os.path.join(inputdir, file)
    output_path = os.path.join(outputdir, file)

    # Skip files that are already processed
    if os.path.exists(output_path):
        print(f"File {file} already processed")
        return file

    # Read the input file and collect texts for classification
    texts = []
    text_count = []
    with open(input_path, 'r') as f:
        for line in f:
            text_entry = json.loads(line)
            count = 0
            for obj in text_entry['output']['Objects']:
                texts.append(obj["text"])
                count += 1
            text_count.append(count)

    # Classify emotions in batch
    batch_results = classifier(texts, batch_size=32)  # Adjust batch size based on GPU memory
    print("Done classifying emotions for file", file)

    # Update the original data with classification results
    text_index = 0
    with open(input_path, 'r') as f, open(output_path, 'w') as output_file:
        for line, count in zip(f, text_count):
            text_entry = json.loads(line)
            for obj in text_entry['output']['Objects']:
                obj["emotion"] = batch_results[text_index]
                text_index += 1
            output_file.write(json.dumps(text_entry) + '\n')
    print(f"File {file} written")
    return file

def main():
    # Get the list of files to process
    files = os.listdir(inputdir)

    # Use ThreadPoolExecutor to process files in parallel
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_file, files))

    print("Processing complete for files:", results)

# Run the main function
main()


File twitter_stream_2020_01_07_filtered.json already processed
File twitter-stream-2021-05-24_filtered.json already processed
File twitter-stream-20211004_filtered.json already processed
File twitter-stream-2020-09-22_filtered.json already processed
File twitter-stream-2020-11-21_filtered.json already processed
File twitter-stream-2020-07-12_filtered.json already processed
File twitter-stream-2020-12-21_filtered.json already processed
File twitter-stream-20211028_filtered.json already processed
File twitter-stream-2021-02-25_filtered.json already processed
File twitter-stream-2020-10-16_filtered.json already processed
File twitter-stream-2021-03-03_filtered.json already processed
File twitter-stream-2021-07-05_filtered.json already processed
File twitter-stream-2020-12-03_filtered.json already processed
File twitter-stream-20211022_filtered.json already processed
File twitter-stream-20211127_filtered.json already processed
File twitter-stream-2021-01-26_filtered.json already processed


In [29]:
# now doing the emotions processing
input_dir = 'data_with_emotions'
output_dir = 'data_with_parent_emotions'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def process_emotions(obj, emots):
    if emots[0]['score'] > 0.75:
        obj['emotion'].append(change_to_parent_emotion(emots[0]))
    else:
        for res in emots:
            if res['label'] == 'neutral' and res['score'] > 0.5:
                obj['emotion'].append(change_to_parent_emotion(res))
            elif (res['label'] != 'neutral' and res['score'] > 0.15):
                obj['emotion'].append(change_to_parent_emotion(res))


for file in os.listdir(input_dir):
    if file in os.listdir(output_dir):
        print(f"File {file} already processed")
        continue
    with open(os.path.join(input_dir, file)) as f1:
        with open(os.path.join(output_dir, file), 'w') as f2:
            for line in f1:
                text1 = json.loads(line)
                for obj in text1['output']['Objects']:
                    # keep only text and emotion in the object
                    for key in list(obj.keys()):
                        if key not in ['text', 'emotion']:
                            del obj[key]
                    emots = obj['emotion'].copy()
                    obj['emotion'] = []
                    process_emotions(obj, emots)
                json.dump(text1, f2)
                f2.write('\n')
    print(f"File {file} written")

File twitter_stream_2020_01_07_filtered.json already processed
File twitter-stream-2021-05-24_filtered.json already processed
File twitter-stream-20211004_filtered.json already processed
File twitter-stream-2020-09-22_filtered.json already processed
File twitter-stream-2020-11-21_filtered.json already processed
File twitter-stream-2020-07-12_filtered.json already processed
File twitter-stream-2020-12-21_filtered.json already processed
File twitter-stream-20211028_filtered.json already processed
File twitter-stream-2021-02-25_filtered.json already processed
File twitter-stream-2020-10-16_filtered.json already processed
File twitter-stream-2021-03-03_filtered.json already processed
File twitter-stream-2021-07-05_filtered.json already processed
File twitter-stream-2020-12-03_filtered.json already processed
File twitter-stream-20211022_filtered.json already processed
File twitter-stream-20211127_filtered.json already processed
File twitter-stream-2021-01-26_filtered.json already processed


In [ ]:
# now making graphs
import matplotlib.pyplot as plt
import seaborn as sns


keywords = [
    "covid-19", 
    "sars", 
    "coronavirus",
    "corona" 
    "covid19", 
    "SARS-CoV-2", 
    "loss of taste", 
    "quarantine", 
    "isolation", 
    "Pfizer", "Moderna", "AstraZeneca", 
]


# Category to emotions mapping
category_to_emotions = {
    "Positive or Joyful Emotions": [
        "admiration", "amusement", "joy", "gratitude", "excitement", 
        "optimism", "pride", "relief"
    ],
    "Negative or Fearful Emotions": [
        "anger", "annoyance", "fear", "nervousness", "sadness", 
        "grief", "remorse"
    ],
    "Neutral or Objective Emotions": [
        "neutral", "approval", "realization"
    ],  
    "Love and Caring Emotions": [
        "love", "caring", "desire"
    ],
    "Surprise or Unexpected Responses": [
        "surprise", "curiosity", "confusion"
    ],
    "Displeasure or Disapproval Emotions": [
        "disappointment", "disapproval", "disgust","embarrassment"
    ],
}

